# Agent dynamic traits
This notebook provides a demonstration of the optional `Agent` parameter `dynamic_traits_function` that can be used to generate agent traits dynamically based on the question being asked or the scenario in which the question is asked. 

**Note:** This method can only be used locally. It is not available with remote inference.

Learn more about this method in the docs: [Agent dynamic traits function](https://docs.expectedparrot.com/en/latest/agents.html#dynamic-traits-function)

## How it works
Agents are created by passing a dictionary of `traits` to an `Agent` object. For example:

In [1]:
from edsl import Agent

my_agent = Agent(
    name="Robin",
    traits={"persona": "You are a middle-aged mom.", "current_vehicle": "minivan"}
)

When we run a survey with this agent, the language model will reference the agent's traits in generating responses. We can test this:

In [2]:
from edsl import QuestionFreeText, Survey

q1 = QuestionFreeText(question_name="age", question_text="How old are you?")
q2 = QuestionFreeText(question_name="car", question_text="What are you driving?")

survey = Survey([q1, q2])

results = survey.by(my_agent).run() # this survey can be run remotely or locally

results.select("agent_name", "answer.*")

Job UUID,d4457aa3-b8ed-47a3-a092-1644a7e3680f
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/d4457aa3-b8ed-47a3-a092-1644a7e3680f
Exceptions Report URL,None
Results UUID,26ed4d6e-d322-4a9e-a370-905e0ac2cec4
Results URL,https://www.expectedparrot.com/content/26ed4d6e-d322-4a9e-a370-905e0ac2cec4


,agent.agent_name,answer.age,answer.car
0,Robin,"Oh, I'm in my mid-40s. Just old enough to have seen my fair share of life, but still young enough to keep up with the kids!","Oh, I drive a minivan. It's perfect for hauling the kids around and fitting all their stuff, plus it's super comfy for road trips!"


## Designing question-based traits
For efficiency or other reasons, we may want to minimize the set of traits that we pass to the agent when we create it, and instead only generate certain traits to use with specific questions. To do this, we can create a method for the desired conditional logic and pass it to an agent as the `dynamic_traits_function` parameter:

In [3]:
# Create a method for the desired logic
def politics_response(question):
    if question.question_name == "politics":
        return {
            "your_politics": "No comment."
        }  # trait to be passed to the agent


# Pass it to the agent
a = Agent(dynamic_traits_function=politics_response)

# Test it 
q = QuestionFreeText(question_name="politics", question_text="What do you think of politics?")

results = q.by(a).run(disable_remote_inference = True) # this question must be run locally

# Inspect the response
results.select("answer.*")

,answer.politics
0,No comment.
